# Recommendation systems
Welcome to your first homework assignment about recommendation systems.


1. Watch this video on Youtube recommendation system https://www.youtube.com/watch?v=zzTbptEdKhY

2. ## The Cold Start Problem 

The colaborative filtering method discussed in class does not address the problem of new user or new movies. What prediction would you use in these cases:

* A new user but a known movie
* A new movie and a known user
* A new user and new movie

3. ## Matrix Factorization with bias
We want to extend the Matrix Factorization model discussed in class to add a "bias" parameter for each user and another "bias" parameter for each movie.  For the problem in class we had the parameters matrix $U$ and $V$, we are adding $u^0$ which is a vector of dimension $n_u$ and $v^0$ which is a vector of dimension $n_m$. The equations

    $\hat{y}_{ij} = u_{0i} + v_{0j} + u_i \cdot v_j  $
 
(a) How many weights (parameters) are we fitting for this problem?

There are $n_u * K + n_m * K + n_m + n_u = {(K+1)}{(n_u + n_m)}$ parameters.

(b) Write the gradient descent equations for this problem.

$u_{is},v_{js},u_{0i}, v_{0j}  \leftarrow $random  initialization

$for \ {{i = 1: maxiter}} \ do:$

$u_{is} \leftarrow u_{is} + \frac{2\eta}{N} \sum_{i, j:r_{ij}=1}(y_{ij}-u_iv_j-u_{0i}-v_{0j})v_{js};$

$v_{js} \leftarrow v_{js} + \frac{2\eta}{N} \sum_{i, j:r_{ij}=1}(y_{ij}-u_iv_j-u_{0i}-v_{0j})u_{is};$

$u_{0i} \leftarrow u_{0i} + \frac{2\eta}{N} \sum_{i, j:r_{ij}=1}(y_{ij}-u_iv_j-u_{0i}-v_{0j});$

$v_{0j} \leftarrow v_{0j} + \frac{2\eta}{N} \sum_{i, j:r_{ij}=1}(y_{ij}-u_iv_j-u_{0i}-v_{0j});$

4. ## Collaborative Filtering with Gradient Descent 

In this part of the assignment you will build a collaborative filtering model to predict netflix ratings.  This assignment will step you through how to do this using stochastic gradient descent. 

**Instructions:**
- Do not use loops (for/while) in your code, unless the instructions explicitly ask you to do so.
- DO NOT change paths (-3 points)
- DO NOT submit data to github (-2 points)

**You will learn to:**
- Build the general architecture of a learning algorithm, including:
    - Encoding rating data
    - Initializing parameters
    - Calculating the cost function
    - Calculating gradient
    - Using an optimization algorithm (gradient descent) 
    - Predicting on new data
- Putting it all together.

In [1]:
import numpy as np
import pandas as pd

## Encoding rating data
Here are our very small subset of fake data to get us started.

In [2]:
# The first row says that user 11 reated movie 1 with a score of 4
!cat tiny_training2.csv 

userId,movieId,rating
11,1,4
11,23,5
2,23,5
2,4,3
31,1,4
31,23,4
4,1,5
4,3,2
52,1,1
52,3,4
61,3,5
7,23,1
7,3,3


In [3]:
# here is a handy function from fast.ai
def proc_col(col):
    """Encodes a pandas column with continous ids. 
    """
    uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx[x] for x in col]), len(uniq)

In [32]:
def encode_data(df):
    """Encodes rating data with continous user and movie ids using 
    the helpful fast.ai function from above.
    
    Arguments:
      train_csv: a csv file with columns user_id,movie_id,rating 
    
    Returns:
      df: a dataframe with the encode data
      num_users
      num_movies
      
    """
    ### BEGIN SOLUTION
    idx_u, array_u, len_u = proc_col(df["userId"])
    idx_m, array_m, len_m = proc_col(df["movieId"])
    num_users = len_u
    num_movies = len_m
    df["userId"] = array_u
    df["movieId"] = array_m
    ### END SOLUTION
    return df, num_users, num_movies

In [33]:
df = pd.read_csv("tiny_training2.csv")
df, num_users, num_movies = encode_data(df)

In [35]:
assert(num_users == 7)

In [36]:
assert(num_movies == 4)

In [37]:
np.testing.assert_equal(df["userId"].values, np.array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 6, 6]))

In [38]:
np.testing.assert_equal(df["movieId"].values, np.array([0, 1, 1, 2, 0, 1, 0, 3, 0, 3, 3, 1, 3]))

## Initializing parameters

In [39]:
def create_embedings(n, K):
    """ Create a numpy random matrix of shape n, K
    
    The random matrix should be initialized with uniform values in (0, 6/K)
    Arguments:
    
    Inputs:
    n: number of items/users
    K: number of factors in the embeding 
    
    Returns:
    emb: numpy array of shape (n, num_factors)
    """
    np.random.seed(3)
    emb = 6*np.random.random((n, K)) / K
    return emb

# here is an example on how the prediction matrix would look like with 7 users and 5 movies
np.dot(create_embedings(7,3), create_embedings(5,3).transpose())

array([[3.55790894, 4.69774849, 0.92361109, 1.58739544, 3.00593239],
       [4.69774849, 7.44656163, 1.18135616, 2.64524868, 4.74559066],
       [0.92361109, 1.18135616, 0.24548062, 0.34025121, 0.69616965],
       [1.58739544, 2.64524868, 0.34025121, 1.61561   , 2.41361975],
       [3.00593239, 4.74559066, 0.69616965, 2.41361975, 3.82505541],
       [2.02000808, 3.29656257, 0.43174569, 2.065911  , 3.07264619],
       [2.07691001, 3.02887291, 0.53270924, 1.02482544, 1.90251125]])

## Encoding Y as a sparse matrix
This code helps you encode a $Y$ as a sparse matrix from the dataframe. 

In [40]:
from scipy import sparse
def df2matrix(df, nrows, ncols, column_name="rating"):
    """ Returns a sparse matrix constructed from a dataframe
    
    This code assumes the df has columns: MovieID,UserID,Rating
    """
    values = df[column_name].values
    ind_movie = df['movieId'].values
    ind_user = df['userId'].values
    return sparse.csc_matrix((values,(ind_user, ind_movie)),shape=(nrows, ncols))

In [286]:
df = pd.read_csv("tiny_training2.csv")
df, num_users, num_movies = encode_data(df)
Y = df2matrix(df, num_users, num_movies)

## Predicting Ratings

In [340]:
def predict(df, emb_user, emb_movie):
    """ This function computes df["prediction"] without doing (U*V^T).

    Compute df["prediction"] by using elementwise multiplication of the corresponding embeddings and then 
    sum to get the prediction u_i*v_j. This avoids creating the dense matrix U*V^T.
    """
    # BEGIN SOLUTION
    userId = df["userId"].astype(int)
    movieId = df["movieId"].astype(int)
    df["prediction"] = np.sum(np.multiply(emb_user[userId], emb_movie[movieId]),axis=1)
    # END SOLUTION
    return df

In [341]:
emb_user = np.ones((num_users, 3))
emb_movie = np.ones((num_movies, 3))
df = predict(df, emb_user, emb_movie)
assert(df["prediction"][12] == 3.0)

In [343]:
emb_user = create_embedings(num_users, K=4)
emb_movie = create_embedings(num_movies, K=4)
df = predict(df, emb_user, emb_movie)
pred_12 = df["prediction"][12]
assert(np.around(pred_12, decimals=2)== 2.05)

## Calculating the cost function

In [344]:
# Use vectorized computation for this function. No loops!
# Hint: use predict function
def cost(df, emb_user, emb_movie):
    """ Computes mean square error
    
    First compute prediction using the predict function.
    Prediction for user i and movie j is emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      error(float): this is the MSE
    """
    ### BEGIN SOLUTION
    df = predict(df, emb_user, emb_movie)
    error = np.mean(np.power(df["prediction"] - df["rating"], 2))
    
    ### END SOLUTION
    return error

In [345]:
emb_user = np.ones((num_users, 3))
emb_movie = np.ones((num_movies, 3))
error = cost(df, emb_user, emb_movie)
assert(np.around(error, decimals=2) == 2.23)

In [346]:
emb_user = create_embedings(num_users, K=4)
emb_movie = create_embedings(num_movies, K=4)
error = cost(df, emb_user, emb_movie)
assert(np.around(error, decimals=2) == 4.36)

## Calculating gradient

In [347]:
def finite_difference(df, emb_user, emb_movie, ind_u=None, ind_m=None, k=None):
    """ Computes finite difference on MSE(U, V).
    
    This function is used for testing the gradient function. 
    """
    e = 0.000000001
    c1 = cost(df, emb_user, emb_movie)
    K = emb_user.shape[1]
    x = np.zeros_like(emb_user)
    y = np.zeros_like(emb_movie)
    if ind_u is not None:
        x[ind_u][k] = e
    else:
        y[ind_m][k] = e
    c2 = cost(df, emb_user + x, emb_movie + y)
    return (c2 - c1)/e

In [348]:
def gradient(df, emb_user, emb_movie):
    """ Computes the gradient.
    
    Hint: First compute df["prediction"]. Then use df2matrix
    to get a sparse matrix Y and Y_hat.
    
    Arguments:
      df: dataframe with all data or a subset of the data
      Y: sparse representation of df
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      d_emb_user
      d_emb_movie
    """
    ### BEGIN SOLUTION
    df = predict(df, emb_user=emb_user, emb_movie=emb_movie)
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    Y_hat = df2matrix(df, emb_user.shape[0], emb_movie.shape[0], column_name="prediction")
    delta = Y - Y_hat
    N = len(df)
    grad_user = -2/N * (delta@emb_movie)
    grad_movie = -2/N * (delta.T@emb_user)
    ### END SOLUTION
    return grad_user, grad_movie

In [349]:
K = 3
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
grad_user, grad_movie = gradient(df, emb_user, emb_movie)

In [350]:
user=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_u=user, k=i) for i in range(K)])
assert(np.all(np.abs(grad_user[user] - approx) < 0.0001))

In [351]:
movie=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_m=movie, k=i) for i in range(K)])
assert(np.all(np.abs(grad_movie[movie] - approx) < 0.0001))

## Using gradient descent with momentum

In [367]:
# you can use a for loop to iterate through gradient descent
def gradient_descent(df, emb_user, emb_movie, iterations=100, learning_rate=0.01, df_val=None):
    """ Computes gradient descent with momentum (0.9) for a number of iterations.

    Prints training cost and validation cost (if df_val is not None) every 50 iterations.

    Returns:
    emb_user: the trained user embedding
    emb_movie: the trained movie embedding
    """
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    # BEGIN SOLUTION

    beta = 0.9

    # Initialize the v and w
    v_user, v_movie = gradient(df, emb_user, emb_movie)
    w_user = emb_user
    w_movie = emb_movie

    # main body: iterate
    for i in range(1, iterations+1):
        grad_user, grad_movie = gradient(df, w_user, w_movie)
        v_user = beta*v_user + (1-beta)*grad_user
        v_movie = beta*v_movie + (1-beta)*grad_movie
        w_user = w_user - learning_rate*v_user
        w_movie = w_movie - learning_rate*v_movie

        if (i % 50 == 0) and (df_val is not None):
            training_cost = cost(df, w_user, w_movie)
            validation_cost = cost(df_val, w_user, w_movie)
            print(f"GD for {i} times", training_cost, validation_cost)

    emb_user = w_user
    emb_movie = w_movie
    
    # END SOLUTION
    return emb_user, emb_movie

In [368]:
emb_user = create_embedings(num_users, 3)
emb_movie = create_embedings(num_movies, 3)
emb_user, emb_movie = gradient_descent(df, emb_user, emb_movie, iterations=200, learning_rate=0.01)

In [369]:
train_mse = cost(df, emb_user, emb_movie)
assert(np.around(train_mse, decimals=2) == 0.53)

## Predicting on new data
Now we should write a function that given new data is able to predict ratings. First we write a function that encodes new data. If a new user or item is present that row should be remove. Collaborative Filtering is not good at handling new users or new items. To help with this task, you could write a an auxiliary function similar to `proc_col`.

In [370]:
def encode_new_data(df_val, df_train):
    """ Encodes df_val with the same encoding as df_train.
    Returns:
    df_val: dataframe with the same encoding as df_train
    """
    ### BEGIN SOLUTION
    idx_u, array_u, len_u = proc_col(df_train["userId"])
    idx_m, array_m, len_m = proc_col(df_train["movieId"])
    
    df_train = encode_data(df_train)[0]
    array_val_u = np.array([idx_u[x] if x in idx_u.keys() else None for x in df_val["userId"]])
    array_val_m = np.array([idx_m[x] if x in idx_m.keys() else None for x in df_val["movieId"]])
    df_val["userId"] = array_val_u
    df_val["movieId"] = array_val_m
    df_val = df_val.dropna()
    ### END SOLUTION
    return df_val

In [356]:
df_t = pd.read_csv("tiny_training2.csv")
df_v = pd.read_csv("tiny_val2.csv")
df_v = encode_new_data(df_v, df_t)

In [357]:
assert(len(df_v.userId.unique())==2)

In [358]:
assert(len(df_v) == 2)

## Putting it all together
For this part you should get data from here
`wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip`

In [371]:
# Don't change this path use a simlink if you have the data somewhere else
path = "ml-latest-small/"
data = pd.read_csv(path + "ratings.csv")
# sorting by timestamp take as validation data the most recent data doesn't work so let's just take 20%
# at random
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8
train = data[msk].copy()
val = data[~msk].copy()
df_train, num_users, num_movies = encode_data(train.copy())
df_val = encode_new_data(val.copy(), train.copy())
print(len(val), len(df_val))

20386 19591


In [361]:
K = 50
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=200, learning_rate=1, df_val=df_val)

12.066111844259689 12.154155894908106
9.423540584309816 9.551439971806266
6.642415798407898 6.7750849635195625
4.776668795933097 4.901094765726285


In [362]:
train_mse = cost(df_train, emb_user, emb_movie)
val_mse = cost(df_val, emb_user, emb_movie)
print(train_mse, val_mse)

3.756714365159779 3.868546473763648


Please check that when you run gradient descent for 2000 iterations. `val_mse` should be around 0.855

In [363]:
train_mse = cost(df_train, emb_user, emb_movie)
assert(np.around(train_mse, decimals=1) == 3.8)

In [364]:
val_mse = cost(df_val, emb_user, emb_movie)
assert(np.around(val_mse, decimals=1) == 3.9)

In [372]:
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=2000, learning_rate=1, df_val=df_val)

GD for 50 times 2.0420569907035064 2.089546019586407
GD for 100 times 1.6789341263553599 1.7271211278551009
GD for 150 times 1.4856983647335988 1.5333151621234973
GD for 200 times 1.3608828396792538 1.4079927189814037
GD for 250 times 1.2717497698713753 1.3187234176767828
GD for 300 times 1.2040922471012516 1.251292996111335
GD for 350 times 1.150578312350897 1.198287869225563
GD for 400 times 1.1069670446496327 1.1553928478240079
GD for 450 times 1.07060376240629 1.1198981011400984
GD for 500 times 1.0397276480430198 1.090003185871876
GD for 550 times 1.0131178826047291 1.0644582120439947
GD for 600 times 0.9898979800186254 1.0423646523826153
GD for 650 times 0.9694206881145128 1.0230580811078118
GD for 700 times 0.9511967907602755 1.0060356756998692
GD for 750 times 0.9348492001710167 0.9909094911103419
GD for 800 times 0.9200823169581198 0.9773752629015918
GD for 850 times 0.9066609813473958 0.9651909470903248
GD for 900 times 0.8943956566842799 0.9541615802847231
GD for 950 times 0